You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Merge Operation:
o	Perform a "merge" (or join) operation between the launches data and the rockets data.
o	Merge Logic: For each launch record, add the rocket's name (rocket_name) from the rockets dataset. The match should be based on the rocket.rocket_id field present in each launch record corresponding to the rocket_id field in each rocket record.
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with its corresponding rocket name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable.
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, number of rockets retrieved, starting merge operation, number of records after merge, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + merge). Print this time after the merge operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting list from the merge operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.

In [0]:
# Databricks notebook source
import requests
import json
import logging
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define API endpoint URLs
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
HTTPBIN_POST_URL = "https://httpbin.org/post"

def fetch_data_from_api(url):
    """
    Fetches data from a given API endpoint.

    Args:
        url (str): The URL of the API endpoint.

    Returns:
        list or None: A list of dictionaries containing the API response data,
                     or None if an error occurred.
    """
    try:
        logging.info(f"Fetching data from: {url}")
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from {url}: {e}")
        return None

def merge_launches_and_rockets(launches, rockets):
    """
    Merges launch data with rocket names based on rocket IDs.

    Args:
        launches (list): A list of launch dictionaries.
        rockets (list): A list of rocket dictionaries.

    Returns:
        list: A list of launch dictionaries enriched with the 'rocket_name'.
    """
    if not launches or not rockets:
        logging.warning("No launches or rockets data available for merging.")
        return []

    rockets_map = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets}
    merged_data = []
    for launch in launches:
        rocket_id = launch.get('rocket', {}).get('rocket_id')
        if rocket_id and rocket_id in rockets_map:
            launch['rocket_name'] = rockets_map[rocket_id]
            merged_data.append(launch)
        else:
            logging.warning(f"Could not find rocket name for launch with ID: {launch.get('flight_number')}")
            # Optionally, you could still include the launch without the rocket name
            # launch['rocket_name'] = None
            # merged_data.append(launch)
    return merged_data

def upload_data_to_httpbin(data):
    """
    Uploads data to the httpbin.org/post endpoint as JSON.

    Args:
        data (list): The data to be uploaded.

    Returns:
        bool: True if the upload was successful, False otherwise.
    """
    try:
        logging.info(f"Uploading data to: {HTTPBIN_POST_URL}")
        headers = {'Content-Type': 'application/json'}
        json_data = json.dumps(data)
        response = requests.post(HTTPBIN_POST_URL, headers=headers, data=json_data, timeout=10)
        response.raise_for_status()
        logging.info(f"Upload to httpbin successful. Status code: {response.status_code}")
        return True
    except requests.exceptions.RequestException as e:
        logging.error(f"Error uploading data to httpbin: {e}")
        return False

if __name__ == "__main__":
    total_start_time = time.time()

    logging.info("Starting data retrieval from SpaceX API.")
    start_time = time.time()
    launches_data = fetch_data_from_api(LAUNCHES_API_URL)
    rockets_data = fetch_data_from_api(ROCKETS_API_URL)
    end_time = time.time()
    retrieval_time = end_time - start_time

    if launches_data:
        logging.info(f"Retrieved {len(launches_data)} launch records.")
    else:
        logging.warning("Failed to retrieve launch data.")

    if rockets_data:
        logging.info(f"Retrieved {len(rockets_data)} rocket records.")
    else:
        logging.warning("Failed to retrieve rocket data.")

    if launches_data and rockets_data:
        logging.info("Starting merge operation.")
        start_time = time.time()
        merged_data = merge_launches_and_rockets(launches_data, rockets_data)
        end_time = time.time()
        merge_time = end_time - start_time
        logging.info(f"Merge operation completed. Number of records after merge: {len(merged_data)}.")
        logging.info(f"Time taken for data retrieval and merge: {retrieval_time + merge_time:.2f} seconds.")

        logging.info("Starting upload to httpbin.")
        upload_successful = upload_data_to_httpbin(merged_data)
        if upload_successful:
            logging.info("Upload to httpbin was successful.")
        else:
            logging.error("Upload to httpbin failed.")
    else:
        logging.warning("Skipping merge and upload due to missing data.")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    logging.info(f"Total execution time of the script: {total_execution_time:.2f} seconds.")